In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from Preprocessing import *
from Models import *
import copy
from Utils import *
pd.set_option('display.max_rows', 200)


In [3]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [41]:
data = DTDataset(use_smote=False)
data.processed_df.T
# data.processed_df#.shape, len(data.processed_df.index.unique())

id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
hpv,1,0,1,1,1,1,-1,1,0,1,...,0,1,-1,0,1,1,0,1,0,1
age,55.969444,20.95,69.930556,72.319444,59.730556,60.083333,67.708333,57.858333,51.758333,56.25,...,47.619444,50.163889,70.888889,67.825,56.336111,49.566667,48.705556,77.116667,45.95,49.733333
packs_per_year,0.0,38.0,35.0,0.0,0.0,0.0,40.0,44.0,0.0,40.0,...,5.0,0.0,50.0,0.0,0.0,30.0,30.0,0.0,5.0,0.0
gender,1,1,0,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,1,1
smoking_status,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.5,0.0,0.5,0.0,0.0,1.0,1.0,0.0,0.5,0.0
Aspiration rate Pre-therapy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
total_dose,66.0,72.0,70.0,70.0,66.0,66.0,69.96,70.0,70.0,70.0,...,70.0,72.0,66.0,70.0,69.96,70.0,72.0,70.0,69.96,69.96
dose_fraction,2.2,1.8,2.121212,2.121212,2.2,2.2,2.12,2.121212,2.0,2.121212,...,2.121212,1.8,2.2,2.121212,2.12,2.121212,1.714286,2.333333,2.12,2.12
OS (Calculated),6.033333,7.333333,7.466667,7.8,8.066667,8.733333,9.1,9.8,10.033333,10.033333,...,139.033333,139.3,140.6,142.833333,143.033333,143.2,144.366667,148.366667,152.6,155.533333
Locoregional control (Time),4.7,7.333333,7.466667,7.8,8.066667,8.733333,6.7,8.5,10.033333,10.033333,...,139.033333,139.3,140.6,142.833333,143.033333,143.2,144.366667,148.366667,152.6,155.533333


In [5]:
def state_loss(ytrue,ypred,subweights=None,weights=None):
    if weights is None:
        weights = [1,1,1,1]
    if subweights is None:
        subweights = [None,None,None]
    pd_loss = torch.mul(mc_loss(ytrue[0],ypred[0],weights=subweights[0]),weights[0])
    nd_loss = torch.mul(mc_loss(ytrue[1],ypred[1],weights=subweights[1]),weights[1])
    mod_loss = torch.mul(mc_loss(ytrue[2],ypred[2],weights=subweights[2]),weights[2])
    loss = torch.add(pd_loss,torch.add(nd_loss,mod_loss))
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        dlt_loss = torch.mul(dlt_loss,weights[3]/ndlt)
        loss = torch.add(loss,dlt_loss)
    return loss

def outcome_loss(ytrue,ypred,weights=None,**kwargs):
    if weights is None:
        weights = [1,1,1,1]
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

In [43]:

        
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.001,
                epochs=1000,
                patience=10,
                weights=None,
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                balanced=True,
                sqrt_balance_weights=False,
                use_smote=False,
                smote_cols = None,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        if smote_cols is None:
            smote_cols = Const.outcomes
            if state == 1:
                smote_cols = Const.primary_disease_states
            elif state == 2:
                smote_cols = Const.primary_disease_states2
        dataset = DTDataset(use_smote=True,smote_ids = train_ids,smote_columns=[Const.decisions[state-1]])
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)
    
    model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
                model = BayesianOutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
        if weights is None:
            weights = [1,1,.1,1]
    else:
        if model is None:
                model = BayesianEndpointSimulator(xtrain.shape[1],**model_args)
        if weights is None:
            weights = [1,1,1,1]
        lfunc = outcome_loss
    print(model.passthrough)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.set_device(device)
    
    balance_weights=None
    if balanced:
        if state < 3:
            balance_weights = [w.to(device) for w in get_weights(ytrain)]
            if sqrt_balance_weights:
                balance_weights = [torch.sqrt(w) for w in balance_weights]
        else:
            print('I dont do balancing on the outputs because Idk how that would work')
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain).to(device)
    
    xtest = df_to_torch(xtest).to(device)
    ytrain = [df_to_torch(t).to(device) for t in ytrain]
    ytest= [df_to_torch(t).to(device) for t in ytest]
    
    model.fit_normalizer(xtrain)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample,n_samples=1)
        loss = lfunc(ytrain,ypred,weights=weights,subweights=balance_weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest,n_samples=1)
        val_loss = lfunc(ytest,yval,weights=weights,subweights=balance_weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest,n_samples=1)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    print(model(xtest))
    return model,  best_val_loss, best_loss_metrics

from Models import *
t1_args = {'hidden_layers': [500,500],
   'dropout': 0.5,
   'input_dropout': 0.1}
tmodel_balanced = train_state(model_args=t1_args,state=1,lr=.0001,weights=[1,1,.1,.1])
tmodel_balanced[-1]

True
epoch 0 train loss 2.4440319538116455
val loss 2.438542366027832
______________
epoch 1 train loss 2.4350571632385254
val loss 2.4368717670440674
______________
epoch 2 train loss 2.44006609916687
val loss 2.4351553916931152
______________
epoch 3 train loss 2.4324288368225098
val loss 2.4334161281585693
______________
epoch 4 train loss 2.429985284805298
val loss 2.4317281246185303
______________
epoch 5 train loss 2.4192652702331543
val loss 2.4300637245178223
______________
epoch 6 train loss 2.420231342315674
val loss 2.4283828735351562
______________
epoch 7 train loss 2.422759532928467
val loss 2.4266951084136963
______________
epoch 8 train loss 2.4105443954467773
val loss 2.424945831298828
______________
epoch 9 train loss 2.410270929336548
val loss 2.423224449157715
______________
epoch 10 train loss 2.408689260482788
val loss 2.421496629714966
______________
epoch 11 train loss 2.4092330932617188
val loss 2.4197540283203125
______________
epoch 12 train loss 2.4065952301

{'pd': {'accuracy': 0.33904761904761904,
  'auc_micro': 0.6888173773129526,
  'auc_mean': 0.5822643737755567,
  'auc_weighted': 0.6178795955864816},
 'nd': {'accuracy': 0.2962962962962963,
  'auc_micro': 0.2262268704746581,
  'auc_mean': 0.4125765318277111,
  'auc_weighted': 0.33426958261863926},
 'mod': {'accuracy': 0.2962962962962963,
  'auc_micro': 0.2262268704746581,
  'auc_mean': 0.4125765318277111,
  'auc_weighted': 0.33426958261863926},
 'dlts': {'accuracy': [0.9464285714285714,
   0.8035714285714286,
   0.8928571428571429,
   0.7857142857142857,
   0.9107142857142857],
  'accuracy_mean': 0.8678571428571429,
  'auc': [0.16981132075471694,
   0.494949494949495,
   0.4933333333333333,
   0.34090909090909094,
   0.7137254901960784],
  'auc_mean': 0.4425457460285429}}

In [44]:
tmodel2_balanced = train_state(model_args=t1_args,state=2,lr=.001,weights=[1,1,.1,.1],use_smote=False)
tmodel2_balanced[-1]

True
epoch 0 train loss 2.387913703918457
val loss 2.3945870399475098
______________
epoch 1 train loss 2.3592090606689453
val loss 2.3996200561523438
______________
epoch 2 train loss 2.3489878177642822
val loss 2.4014124870300293
______________
epoch 3 train loss 2.313110828399658
val loss 2.401705741882324
______________
epoch 4 train loss 2.2768259048461914
val loss 2.400881052017212
______________
epoch 5 train loss 2.2325520515441895
val loss 2.3997104167938232
______________
epoch 6 train loss 2.1908929347991943
val loss 2.3981380462646484
______________
epoch 7 train loss 2.1599292755126953
val loss 2.3965439796447754
______________
epoch 8 train loss 2.0965347290039062
val loss 2.3951566219329834
______________
epoch 9 train loss 2.075035572052002
val loss 2.3927695751190186
______________
epoch 10 train loss 2.006511926651001
val loss 2.3917644023895264
______________
epoch 11 train loss 1.965780258178711
val loss 2.393056869506836
______________
epoch 12 train loss 1.9299523

{'pd': {'accuracy': 0.47024861552963354,
  'auc_micro': 0.9096754582865694,
  'auc_mean': 0.5869738348803483,
  'auc_weighted': 0.6969305375980391},
 'nd': {'accuracy': 0.5175808720112517,
  'auc_micro': 0.6283687943262412,
  'auc_mean': 0.6632516181700389,
  'auc_weighted': 0.5826053414597784},
 'mod': {'accuracy': 0.5175808720112517,
  'auc_micro': 0.6283687943262412,
  'auc_mean': 0.6632516181700389,
  'auc_weighted': 0.5826053414597784},
 'dlts': {'accuracy': [0.9795918367346939,
   0.9183673469387755,
   0.9659863945578231,
   0.9523809523809523,
   0.9659863945578231],
  'accuracy_mean': 0.9564625850340136,
  'auc': [0.7291666666666667,
   0.5487654320987654,
   0.6380281690140845,
   0.6030612244897959,
   0.504225352112676],
  'auc_mean': 0.6046493688763978}}

In [45]:
tmodel1 = train_state(model_args=t1_args,state=1,lr=.0001,weights=[1,1,.1,.1],balanced=False)
tmodel1[1]

True
epoch 0 train loss 2.44364857673645
val loss 2.4350075721740723
______________
epoch 1 train loss 2.4363391399383545
val loss 2.427032709121704
______________
epoch 2 train loss 2.428187847137451
val loss 2.4190123081207275
______________
epoch 3 train loss 2.4175174236297607
val loss 2.410942316055298
______________
epoch 4 train loss 2.410747528076172
val loss 2.402775526046753
______________
epoch 5 train loss 2.4030158519744873
val loss 2.3945116996765137
______________
epoch 6 train loss 2.392439365386963
val loss 2.3861420154571533
______________
epoch 7 train loss 2.38956880569458
val loss 2.3776676654815674
______________
epoch 8 train loss 2.3741800785064697
val loss 2.3690567016601562
______________
epoch 9 train loss 2.3644676208496094
val loss 2.360257148742676
______________
epoch 10 train loss 2.3587591648101807
val loss 2.3513026237487793
______________
epoch 11 train loss 2.3510098457336426
val loss 2.34214186668396
______________
epoch 12 train loss 2.339078903198

val loss 1.8623212575912476
______________
epoch 103 train loss 1.7565207481384277
val loss 1.8617914915084839
______________
epoch 104 train loss 1.76929771900177
val loss 1.861263394355774
______________
epoch 105 train loss 1.7716108560562134
val loss 1.8607293367385864
______________
epoch 106 train loss 1.7784855365753174
val loss 1.8602421283721924
______________
epoch 107 train loss 1.7644717693328857
val loss 1.8597432374954224
______________
epoch 108 train loss 1.764111042022705
val loss 1.8592020273208618
______________
epoch 109 train loss 1.7724149227142334
val loss 1.8587101697921753
______________
epoch 110 train loss 1.7603375911712646
val loss 1.8581945896148682
______________
epoch 111 train loss 1.7681528329849243
val loss 1.8576592206954956
______________
epoch 112 train loss 1.761465072631836
val loss 1.8571878671646118
______________
epoch 113 train loss 1.7547214031219482
val loss 1.8568817377090454
______________
epoch 114 train loss 1.7627179622650146
val loss 

{'pd': {'accuracy': 0.42904761904761907,
  'auc_micro': 0.8299275945293645,
  'auc_mean': 0.7211716825926885,
  'auc_weighted': 0.7375856917444006},
 'nd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8780370072405471,
  'auc_mean': 0.6008165380217266,
  'auc_weighted': 0.6356346483704974},
 'mod': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8780370072405471,
  'auc_mean': 0.6008165380217266,
  'auc_weighted': 0.6356346483704974},
 'dlts': {'accuracy': [0.9464285714285714,
   0.8035714285714286,
   0.8928571428571429,
   0.7857142857142857,
   0.9107142857142857],
  'accuracy_mean': 0.8678571428571429,
  'auc': [0.5408805031446541,
   0.5878787878787879,
   0.36,
   0.4602272727272727,
   0.6745098039215686],
  'auc_mean': 0.5246992735344567}}

In [46]:
tmodel2 = train_state(model_args=t1_args,state=2,lr=.001,weights=[1,1,.1,.1],balanced=False)
tmodel2[-1]

True
epoch 0 train loss 2.387779474258423
val loss 2.2927165031433105
______________
epoch 1 train loss 2.292802333831787
val loss 2.1896159648895264
______________
epoch 2 train loss 2.1866068840026855
val loss 2.0933775901794434
______________
epoch 3 train loss 2.0873255729675293
val loss 2.0109028816223145
______________
epoch 4 train loss 2.0009748935699463
val loss 1.9453884363174438
______________
epoch 5 train loss 1.9259860515594482
val loss 1.8972011804580688
______________
epoch 6 train loss 1.879790186882019
val loss 1.8651847839355469
______________
epoch 7 train loss 1.8379364013671875
val loss 1.8474427461624146
______________
epoch 8 train loss 1.8184651136398315
val loss 1.8404093980789185
______________
epoch 9 train loss 1.800204873085022
val loss 1.8406853675842285
______________
epoch 10 train loss 1.7865368127822876
val loss 1.8426125049591064
______________
epoch 11 train loss 1.7765361070632935
val loss 1.8409820795059204
______________
epoch 12 train loss 1.781

{'pd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8964763374485596,
  'auc_mean': 0.6758263333609268,
  'auc_weighted': 0.6121107083171943},
 'nd': {'accuracy': 0.3581953911067835,
  'auc_micro': 0.7434988179669031,
  'auc_mean': 0.49074196972765854,
  'auc_weighted': 0.5207666475505085},
 'mod': {'accuracy': 0.3581953911067835,
  'auc_micro': 0.7434988179669031,
  'auc_mean': 0.49074196972765854,
  'auc_weighted': 0.5207666475505085},
 'dlts': {'accuracy': [0.9795918367346939,
   0.9183673469387755,
   0.9659863945578231,
   0.9523809523809523,
   0.9659863945578231],
  'accuracy_mean': 0.9564625850340136,
  'auc': [0.673611111111111,
   0.587037037037037,
   0.7211267605633803,
   0.6846938775510204,
   0.4971830985915493],
  'auc_mean': 0.6327303769708197}}

In [47]:
tmodel3 = train_state(model_args=t1_args,state=3,lr=.0001,weights=[1,5,10,.11],balanced=False)
tmodel3[-1]

True
epoch 0 train loss 11.259885787963867
val loss 11.07091999053955
______________
epoch 1 train loss 11.018091201782227
val loss 10.88207721710205
______________
epoch 2 train loss 10.847399711608887
val loss 10.704755783081055
______________
epoch 3 train loss 10.664331436157227
val loss 10.538002967834473
______________
epoch 4 train loss 10.456486701965332
val loss 10.380908966064453
______________
epoch 5 train loss 10.251359939575195
val loss 10.232665061950684
______________
epoch 6 train loss 10.110241889953613


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 10.093234062194824
______________
epoch 7 train loss 9.955201148986816
val loss 9.961740493774414
______________
epoch 8 train loss 9.786827087402344
val loss 9.838542938232422
______________
epoch 9 train loss 9.700526237487793
val loss 9.722639083862305
______________
epoch 10 train loss 9.528663635253906
val loss 9.61361312866211
______________
epoch 11 train loss 9.406234741210938
val loss 9.511004447937012
______________
epoch 12 train loss 9.348855972290039
val loss 9.414334297180176
______________
epoch 13 train loss 9.201764106750488


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 9.323132514953613
______________
epoch 14 train loss 9.100842475891113
val loss 9.237297058105469
______________
epoch 15 train loss 8.963264465332031
val loss 9.156431198120117
______________
epoch 16 train loss 8.846184730529785
val loss 9.080273628234863
______________
epoch 17 train loss 8.829504013061523
val loss 9.00875473022461
______________
epoch 18 train loss 8.699515342712402
val loss 8.941627502441406
______________
epoch 19 train loss 8.594953536987305
val loss 8.87819766998291
______________
epoch 20 train loss 8.542309761047363


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 6.712054252624512
______________
epoch 104 train loss 5.48576545715332
val loss 6.707007884979248
______________
epoch 105 train loss 5.416004180908203
val loss 6.702162742614746
______________
epoch 106 train loss 5.330256462097168
val loss 6.697447776794434
______________
epoch 107 train loss 5.322304725646973
val loss 6.692955493927002
______________
epoch 108 train loss 5.353653430938721
val loss 6.688661575317383
______________
epoch 109 train loss 5.380661487579346
val loss 6.684647560119629
______________
epoch 110 train loss 5.364022731781006


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


val loss 6.68086051940918
______________
epoch 111 train loss 5.272409915924072
val loss 6.67742395401001
______________
epoch 112 train loss 5.366327285766602
val loss 6.674243450164795
______________
epoch 113 train loss 5.319035530090332
val loss 6.671271800994873
______________
epoch 114 train loss 5.161516189575195
val loss 6.66888427734375
______________
epoch 115 train loss 5.268605709075928
val loss 6.666612148284912
______________
epoch 116 train loss 5.19114351272583
val loss 6.664812088012695
______________
epoch 117 train loss 5.317132472991943
val loss 6.6635026931762695
______________
epoch 118 train loss 5.195349216461182
val loss 6.662083148956299
______________
epoch 119 train loss 5.130294322967529
val loss 6.661245822906494
______________
epoch 120 train loss 5.184659957885742
val loss 6.660593509674072
______________
epoch 121 train loss 5.132582187652588
val loss 6.659907341003418
______________
epoch 122 train loss 5.158051490783691
val loss 6.6588616371154785
___

{'Overall Survival (4 Years)': {'accuracy': 0.891156462585034,
  'mse': 0.10611311,
  'auc': 0.4804389312977099,
  'precision': 0.891156462585034,
  'recall': 1.0,
  'f1': 0.9424460431654677},
 'FT': {'accuracy': 0.8027210884353742,
  'mse': 0.16054522,
  'auc': 0.7040760869565218,
  'precision': 0.7142857142857143,
  'recall': 0.15625,
  'f1': 0.25641025641025644},
 'Aspiration rate Post-therapy': {'accuracy': 0.8163265306122449,
  'mse': 0.12298099,
  'auc': 0.8204068658614113,
  'precision': 0.45454545454545453,
  'recall': 0.19230769230769232,
  'f1': 0.27027027027027023},
 'LRC': {'accuracy': 0.8979591836734694,
  'mse': 0.09397956,
  'auc': 0.6681818181818182,
  'precision': 0.8979591836734694,
  'recall': 1.0,
  'f1': 0.9462365591397849}}

In [48]:
tmodel3_balanced = train_state(model_args=t1_args,state=3,lr=.001,weights=[.1,1,10,.1])
tmodel3_balanced[-1]

True
I dont do balancing on the outputs because Idk how that would work
epoch 0 train loss 7.4047651290893555
val loss 6.264412879943848
______________
epoch 1 train loss 6.144690990447998
val loss 5.964177131652832
______________
epoch 2 train loss 5.617372989654541
val loss 5.961685657501221
______________
epoch 3 train loss 5.6885223388671875
val loss 5.889715194702148
______________
epoch 4 train loss 5.4898858070373535
val loss 5.6938629150390625
______________
epoch 5 train loss 5.354703426361084
val loss 5.425809383392334
______________


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

epoch 6 train loss 5.066502094268799
val loss 5.170238971710205
______________
epoch 7 train loss 4.691295623779297
val loss 4.977500915527344
______________
epoch 8 train loss 4.482172966003418
val loss 4.849785327911377
______________
epoch 9 train loss 4.428045749664307
val loss 4.762908935546875
______________
epoch 10 train loss 4.39070463180542
val loss 4.69351053237915
______________
epoch 11 train loss 4.171168804168701
val loss 4.624512672424316
______________
epoch 12 train loss 4.144720554351807


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 4.558403968811035
______________
epoch 13 train loss 3.9548819065093994
val loss 4.506914138793945
______________
epoch 14 train loss 3.9416942596435547
val loss 4.480982303619385
______________
epoch 15 train loss 3.761516571044922
val loss 4.484621524810791
______________
epoch 16 train loss 3.574589967727661
val loss 4.510415554046631
______________
epoch 17 train loss 3.5759263038635254
val loss 4.541672229766846
______________
epoch 18 train loss 3.5946831703186035
val loss 4.554272651672363
______________
epoch 19 train loss 3.488781213760376
val loss 4.545094013214111
______________
epoch 20 train loss 3.3921847343444824
val loss 4.518486022949219
______________
epoch 21 train loss 3.468644380569458
val loss 4.485116958618164
______________
epoch 22 train loss 3.2474606037139893
val loss 4.459201812744141
______________
epoch 23 train loss 3.0273168087005615


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 4.449732780456543
______________
epoch 24 train loss 3.1528425216674805
val loss 4.451713562011719
______________
epoch 25 train loss 3.0233190059661865
val loss 4.466689109802246
______________
epoch 26 train loss 3.034191131591797
val loss 4.492293834686279
______________
epoch 27 train loss 2.8552305698394775
val loss 4.533221244812012
______________
epoch 28 train loss 2.8459911346435547
val loss 4.5962934494018555
______________
epoch 29 train loss 2.7636635303497314
val loss 4.673311710357666
______________
epoch 30 train loss 2.747400999069214
val loss 4.750593185424805
______________
epoch 31 train loss 2.6612210273742676
val loss 4.804327011108398
______________
epoch 32 train loss 2.5833067893981934
val loss 4.828226089477539
______________
epoch 33 train loss 2.3568320274353027
val loss 4.835818290710449
______________
epoch 34 train loss 2.33154559135437
val loss 4.845974922180176
______________
best loss 4.449732780456543 {'Overall Survival (4 Years)': {'accuracy'

{'Overall Survival (4 Years)': {'accuracy': 0.891156462585034,
  'mse': 0.105403766,
  'auc': 0.5353053435114503,
  'precision': 0.891156462585034,
  'recall': 1.0,
  'f1': 0.9424460431654677},
 'FT': {'accuracy': 0.7687074829931972,
  'mse': 0.16273856,
  'auc': 0.7133152173913043,
  'precision': 0.42857142857142855,
  'recall': 0.1875,
  'f1': 0.26086956521739124},
 'Aspiration rate Post-therapy': {'accuracy': 0.8095238095238095,
  'mse': 0.12899455,
  'auc': 0.821360457724094,
  'precision': 0.45454545454545453,
  'recall': 0.38461538461538464,
  'f1': 0.41666666666666663},
 'LRC': {'accuracy': 0.8979591836734694,
  'mse': 0.08835808,
  'auc': 0.6828282828282829,
  'precision': 0.8979591836734694,
  'recall': 1.0,
  'f1': 0.9462365591397849}}

In [50]:
[m[1] for m in [tmodel1,tmodel_balanced,tmodel2,tmodel2_balanced,tmodel3,tmodel3_balanced]]

[1.853824496269226,
 2.3433570861816406,
 1.8347151279449463,
 2.3917644023895264,
 6.651187419891357,
 4.449732780456543]

In [51]:
tmodel1[0].set_device('cpu')
tmodel2[0].set_device('cpu')


torch.save(tmodel1[0],'../resources/transition1_model.pt')
torch.save(tmodel2[0],'../resources/transition2_model.pt')

# tmodel3[0].set_device('cpu')
# torch.save(tmodel3[0],'../resources/outcome_model.pt')
tmodel3_balanced[0].set_device('cpu')
torch.save(tmodel3_balanced[0],'../resources/outcome_model.pt')

In [39]:
test =torch.empty((10,10))
test[:,test.shape[1]-3:]

tensor([[1.6255e-43, 1.6956e-43, 5.6052e-44],
        [5.7453e-44, 1.4013e-44, 1.6255e-43],
        [1.4153e-43, 1.6115e-43, 1.6255e-43],
        [6.8664e-44, 1.3032e-43, 6.3058e-44],
        [0.0000e+00, 2.6627e+07, 4.5744e-41],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0926e-34,        nan,        nan],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.5745e-41, 7.2468e-36, 4.5745e-41]])

In [ ]:
#outdated code for doing an ensembel of stuff instead of using bayesian inference via dropout
def train_model_ensemble(model_arg_list,state,
                         n_bags=20,
                         smote_options=[True,False],
                         smote_baseline_options=[True,False],
                         weight_options=[None],
                         weight_baseline_options=[None],
                         **kwargs):
    resampled_models =[]
    base_models = []
    base_metrics = []
    resample_metrics = []
    base_losses = []
    resample_losses = []
    n_errors = 0
    for margs in model_arg_list:
        for s in smote_baseline_options:
            for w in weight_baseline_options:
                [base_model,blosses,bmetrics] = train_state(model_args=margs,state=state,
                                         resample_training=False,
                                         use_smote=smote_options,
                                         weights=w,
                                         verbose=False,**kwargs)
                base_models.append(base_model)
                base_metrics.append(bmetrics)
                base_losses.append(blosses)
        for n in range(n_bags):
            for s in smote_options:
                for w in weight_options:
                    #this can fail if I resample a bad distribution with no minority classes
                    done = False
                    while not done:
                        try:
                            [model,loss,metrics] = train_state(model_args=margs,
                                                state=state,resample_training=True,
                                                use_smote=s,
                                                weights=w,
                                                verbose=False,**kwargs)
                            resampled_models.append(model)
                            resample_metrics.append(metrics)
                            resample_losses.append(loss)
                            done=True
                            print('model_done',len(base_models)+len(resampled_models))
                        except Exception as e:
                            print('error training model')
                            print(e)
                            n_errors += 1
    print('done with',n_errors,'errors')
    return base_models,resampled_models